## LSTM version, same data

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import tensorflow as tf

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

Using TensorFlow backend.


In [3]:
seed = 123

negative_df = pd.read_csv("fakenews_big.csv", dtype='str')
# negative_df2 = pd.read_csv("fakenews_big2.csv", dtype='str')

positive_df = pd.read_csv("radiocanada.csv", dtype='str')
positive_df.columns = positive_df.ix[0]
positive_df = positive_df[1:]
positive_df = positive_df.sample(n=(negative_df.shape[0]), random_state=seed, axis=0)

print(positive_df.shape)
print(negative_df.shape)
# print(negative_df2.shape)


X_positive = positive_df["text"]
y_positive = pd.DataFrame(np.ones(positive_df.shape[0]))

X_negative = negative_df["text"]
y_negative = pd.DataFrame(np.zeros(negative_df.shape[0]))

# X_negative2 = negative_df2["text"]
# y_negative2 = pd.DataFrame(np.zeros(negative_df2.shape[0]))


train_x = pd.concat([X_positive, X_negative], axis=0)
train_y = pd.concat([y_positive, y_negative], axis=0)

# train_x = train_x.as_matrix().astype('str')

print("X Shape:", train_x.shape)
print("y shape:", train_y.shape)

(2436, 7)
(2436, 6)
X Shape: (4872,)
y shape: (4872, 1)


In [4]:
train_x = train_x.astype('str')
train_x = train_x.apply(lambda x: x.lower())
train_x = train_x.apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_x.values)
X = tokenizer.texts_to_sequences(train_x.values)
X = pad_sequences(X)

In [5]:
embed_dim = 128
lstm_out = 196

import keras

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(keras.layers.SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, recurrent_dropout=0.2, dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4265, 128)         256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 4265, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
Y = to_categorical(num_classes=2,y=train_y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4384, 4265) (4384, 2)
(488, 4265) (488, 2)


In [7]:
batch_size = 32
model.fit(X, 
          Y.astype('int32'),
          epochs = 7, 
          batch_size=64, 
          verbose = 1,
          validation_split=0.1,
          shuffle=True)

Train on 4384 samples, validate on 488 samples
Epoch 1/7
4384/4384 [==============================] - 1326s 303ms/step - loss: 0.7199 - acc: 0.6667 - val_loss: 0.4462 - val_acc: 0.8156
Epoch 2/7
4384/4384 [==============================] - 1326s 303ms/step - loss: 0.2092 - acc: 0.9304 - val_loss: 0.1345 - val_acc: 0.9467
Epoch 3/7
4384/4384 [==============================] - 1275s 291ms/step - loss: 0.1127 - acc: 0.9674 - val_loss: 0.1660 - val_acc: 0.9549
Epoch 4/7
4384/4384 [==============================] - 1274s 291ms/step - loss: 0.0690 - acc: 0.9797 - val_loss: 0.1214 - val_acc: 0.9652
Epoch 5/7
4384/4384 [==============================] - 1274s 291ms/step - loss: 0.0401 - acc: 0.9895 - val_loss: 0.1574 - val_acc: 0.9508
Epoch 6/7
4384/4384 [==============================] - 1280s 292ms/step - loss: 0.0359 - acc: 0.9906 - val_loss: 0.1522 - val_acc: 0.9570
Epoch 7/7
4384/4384 [==============================] - 1273s 290ms/step - loss: 0.1452 - acc: 0.9651 - val_loss: 0.2009 - val

In [9]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

ValueError: not enough values to unpack (expected 2, got 0)

In [22]:
model.predict(X[3990:4000].reshape(10,-1))

array([[ 0.96521139,  0.03478859],
       [ 0.9962759 ,  0.00372411],
       [ 0.95351911,  0.04648097],
       [ 0.94095552,  0.05904449],
       [ 0.96051413,  0.03948583],
       [ 0.99263191,  0.00736807],
       [ 0.98367554,  0.01632441],
       [ 0.99592292,  0.00407704],
       [ 0.91474634,  0.08525373],
       [ 0.99637187,  0.00362819]], dtype=float32)

In [23]:
weights = model.layers[0].get_weights()[0]

In [25]:
weights

array([[-0.04415981,  0.04059046,  0.05120241, ..., -0.00400383,
        -0.00778637,  0.07141628],
       [ 0.01883192, -0.01000453, -0.03343786, ..., -0.03896597,
        -0.04408007,  0.0428776 ],
       [ 0.04448967, -0.0601532 ,  0.00266957, ..., -0.00145122,
        -0.06207993,  0.01591503],
       ..., 
       [ 0.00325109,  0.05314196,  0.03210619, ...,  0.00307486,
        -0.01667879,  0.03583912],
       [ 0.03466699, -0.00568302, -0.01199693, ..., -0.00114606,
         0.06601872,  0.01537393],
       [ 0.03630057, -0.0368008 , -0.01441452, ..., -0.06005743,
        -0.04349767,  0.00222369]], dtype=float32)

In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")